In [14]:

import pandas as pd
import math
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, as_completed
import time
import argparse
import sympy
from functools import lru_cache




@lru_cache(maxsize=None)
def is_prime(num):
    return sympy.isprime(num)


def addition(number_to_generate_primes):
    """Generate primes based on the given number."""
    quotient = number_to_generate_primes
    secondmult = 0
    thirdmult = 0
    primes = []
    multiple_minus_one = 0
    twins_number = 0
    possible_twin1=0
    possible_twin2=0
    primes_plus_one=[]
    primes_minus_one=[]
    primes_plus_one_multiplier=[]
    primes_minus_one_multiplier=[]
    for index in range(2, number_to_generate_primes + 1):
        if((index & 1) == 0 or (quotient & 1) == 0):
            prime_plus_one = index * quotient + 1
            prime_minus_one = index * quotient - 1
            prime1=False
            prime2=False
            if (prime_plus_one&1==1 and is_prime(prime_plus_one)) or prime_plus_one==2:
                secondmult += 1
                prime1=True
                possible_twin1=prime_plus_one
                primes.append(prime_plus_one)
                primes_plus_one.append(prime_plus_one)
                primes_plus_one_multiplier.append(index)

            if (prime_minus_one&1==1 and is_prime(prime_minus_one)) or prime_minus_one==2:
                multiple_minus_one += 1
                prime2=True
                possible_twin2=prime_minus_one
                primes.append(prime_minus_one)
                primes_minus_one.append(prime_minus_one)
                primes_minus_one_multiplier.append(index)
            if(prime1==False or prime2==False):
                continue
            if((prime_plus_one-1) % 6!=0):
                continue
            if  (prime_plus_one-1) %6 ==0 and prime1 and prime2 and (possible_twin1 - possible_twin2 == 2):#all twin except 3,5 the composed number between the is multiple of 6n
                twins_number += 1# the 6n verification used for faster exit if not
    plus_one_csv=f"{number_to_generate_primes}+1"
    minus_one_csv=f"{number_to_generate_primes}-1"

    return secondmult, primes, multiple_minus_one, twins_number,primes_plus_one,primes_minus_one,primes_plus_one_multiplier,primes_minus_one_multiplier


def calculate_range(start, end):
    """Calculate prime-related data for a range of numbers."""
    results = []
    for number in range(start, end + 1):
        result = addition(number)
        prime_check = is_prime(number)
        results.append((number, result, prime_check))
    return results

def calculate(min_interval, end_interval):
    """Perform the main calculations and generate the data for plotting."""
    # Parallel processing
    chunk_size = 10  # Adjust the chunk size based on your system's capabilities
    ranges = [(i, min(i + chunk_size - 1, end_interval)) for i in range(min_interval, end_interval + 1, chunk_size)]
    print(ranges)
    data_to_csv={}
    data_multiplier_to_csv={}
    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(calculate_range, start, end) for start, end in ranges]
        for future in as_completed(futures):
            results = future.result()
            for number, result, prime_check in results:
                if number > 0:  # Ensure number is positive
                    print(number," start")
                    plus_one_csv=f"{number}_plus_1"
                    minus_one_csv=f"{number}_minus_1"
                    data_to_csv[plus_one_csv]=result[4]
                    data_to_csv[minus_one_csv]=result[5]
                    data_multiplier_to_csv[plus_one_csv]=result[6]
                    data_multiplier_to_csv[minus_one_csv]=result[7]
                    print(number," end")




            # Create a DataFrame
    max_length = max(len(arr) for arr in data_to_csv.values())

    # Fill shorter arrays with zeros
    for key in data_to_csv:
        data_to_csv[key].extend([0] * (max_length - len(data_to_csv[key])))
    for key in data_multiplier_to_csv:
        data_multiplier_to_csv[key].extend([0] * (max_length - len(data_multiplier_to_csv[key])))



    df_primes = pd.DataFrame(data_to_csv).fillna(0)
    df = pd.DataFrame(data_multiplier_to_csv).fillna(0)

    # Write the DataFrame to a CSV file
    df_primes.to_csv(f'{min_interval} to {end_interval}primes_each_number.csv', index=False)
    df.to_csv(f'{min_interval} to {end_interval} primes_multiplier_each_number.csv', index=False)
    # Plotting

def count(start,end):
    print("counting started")


    file_path = f'{start} to {end} primes_multiplier_each_number.csv'
    df = pd.read_csv(file_path)


    # Step 2: Count occurrences
    counts = {}

    # Iterate over each column in the DataFrame
    for column in df.columns:
        values = df[column].dropna().tolist()

        for value in values:
            if value not in counts:
                counts[value] = 0
            counts[value] += 1

    # Convert the counts dictionary to a DataFrame
    counts_df = pd.DataFrame(list(counts.items()), columns=['Value', 'Count']).sort_values(by='Value')
    # Display the first few rows of the counts DataFrame
    #print(counts_df.head())
    # Step 3: Save the results
    counts_df.to_csv(f'{start} to {end}  counts .csv', index=False)

def setup(start,end):
    calculate(start, end)
    count(start,end)


def main():
    #min_interval = int(input("Enter the minimum interval: "))
    #end_interval = int(input("Enter the maximum interval: "))
    start_time = time.time()
    start=26001
    end=28000
    setup(start,end)
    end_time = time.time()
    print(f"Elapsed time: {end_time - start_time} seconds")
    

if __name__ == "__main__":
    main()
    

[(26001, 26010), (26011, 26020), (26021, 26030), (26031, 26040), (26041, 26050), (26051, 26060), (26061, 26070), (26071, 26080), (26081, 26090), (26091, 26100), (26101, 26110), (26111, 26120), (26121, 26130), (26131, 26140), (26141, 26150), (26151, 26160), (26161, 26170), (26171, 26180), (26181, 26190), (26191, 26200), (26201, 26210), (26211, 26220), (26221, 26230), (26231, 26240), (26241, 26250), (26251, 26260), (26261, 26270), (26271, 26280), (26281, 26290), (26291, 26300), (26301, 26310), (26311, 26320), (26321, 26330), (26331, 26340), (26341, 26350), (26351, 26360), (26361, 26370), (26371, 26380), (26381, 26390), (26391, 26400), (26401, 26410), (26411, 26420), (26421, 26430), (26431, 26440), (26441, 26450), (26451, 26460), (26461, 26470), (26471, 26480), (26481, 26490), (26491, 26500), (26501, 26510), (26511, 26520), (26521, 26530), (26531, 26540), (26541, 26550), (26551, 26560), (26561, 26570), (26571, 26580), (26581, 26590), (26591, 26600), (26601, 26610), (26611, 26620), (26621,

In [10]:
def count(start=20001,end=22000):
    print("counting started")
    file_path = f'{start} to {end} primes_multiplier_each_number.csv'
    df = pd.read_csv(file_path)


    # Step 2: Count occurrences
    counts = {}

    # Iterate over each column in the DataFrame
    for column in df.columns:
        values = df[column].dropna().tolist()

        for value in values:
            if value not in counts:
                counts[value] = 0
            counts[value] += 1

    # Convert the counts dictionary to a DataFrame
    counts_df = pd.DataFrame(list(counts.items()), columns=['Value', 'Count']).sort_values(by='Value')

    # Display the first few rows of the counts DataFrame
    #print(counts_df.head())
    # Step 3: Save the results
    counts_df.to_csv(f'{start} to {end}  counts .csv', index=False)
count()

counting started
